# Main

In [ ]:
%matplotlib widget

import os
import csv
import time
import psutil
import pyvisa
import random
import threading
import functools
import numpy as np
import pandas as pd
import IPython.display
import ipywidgets as widgets
from datetime import datetime
import matplotlib.pyplot as plt
from kcubedriver import *
from jupyterplot import ProgressPlot

plt.rcParams['figure.autolayout'] = True; plt.rc('font', size=16)
plt.rc('text', usetex=False); plt.rc('font', family='serif'); plt.rcParams['figure.figsize'] = (10, 5)

## init params
global pmHandler, tlsHandler, psHandler, instHandler
resMan = pyvisa.ResourceManager()
outputMain = widgets.Output()

if len(resMan.list_resources()) > 1: instAddress = list(resMan.list_resources())[0] ## ignore if nothing found, can "simulate" from personal laptop
instHandler, tlsHandler, pmHandler, psHandler, switchHandler = 'None', 'None', 'None', 'None', 'None'
sweepStatus, connStatus = 'Idle', 'Idle'

cpuVal = -300; contMeasStatus, pmPortCount = 0, 0
dataTrans = []; dataTransReal = [0, 0, 0, 0, 0, 0, 0, 0]

leftNanoTrakAddr = b'57328344'
leftPiezoAddr = b'29251813'
leftStrainGaugeAddr = b'59001247'
rightNanoTrakAddr = b'57328354'
rightPiezoAddr = b'29251579'
rightStrainGaugeAddr = b'59001269'

tlsAddrUltimae = 'TCPIP0::10.10.21.54::inst0::INSTR'
tlsAddrChariot = 'TCPIP0::10.10.20.173::inst0::INSTR'
pmAddrUltimae = 'TCPIP0::10.10.21.20::inst0::INSTR'
pmAddrChariot = 'TCPIP0::10.10.20.154::inst0::INSTR'
psAddr = 'TCPIP0::10.10.20.224::inst0::INSTR'
# switchAddr = 'TCPIP0::10.10.21.74::inst0::INSTR'
switchAddr = 0

## init widgets
scanButton = widgets.Button(description='Scan for Instruments'); checkConnectButton = widgets.Button(description='Check Connection')
connectInstMeasButton = widgets.Button(description='Connect'); stopMeasButton = widgets.Button(description='Stop & Disconnect')
instIDBox = widgets.Text(); dropdownBox = widgets.Dropdown(options=list(resMan.list_resources()), description='Address'); 
tlsConnBox = widgets.Text(description='Tunable Laser Source Status:', value='Not Connected', style={'description_width':'initial'})
pmConnBox = widgets.Text(description='Power Meter Status:', value='Not Connected', style={'description_width':'initial'})
psConnBox = widgets.Text(description='Polarization Synthesizer Status:', value='Not Connected', style={'description_width':'initial'})
connectChariotButton = widgets.Button(description='Chariot'); connectUltimaeButton = widgets.Button(description='Ultimae')
connectPSCheck = widgets.Checkbox(description='Connect PS?', value=False, indent=False)

## scan and connect
scanConnectButtonHandler = widgets.HBox([scanButton, checkConnectButton])
instFindHandler = widgets.VBox([scanConnectButtonHandler, dropdownBox])
instStartHandler = widgets.VBox([instIDBox, widgets.HBox([connectInstMeasButton, stopMeasButton])])

## first accordion 
cpuUsageBox = widgets.Text(description='CPU Usage :')
instConnHandler = widgets.HBox([widgets.VBox([tlsConnBox, pmConnBox, psConnBox]), cpuUsageBox])

## second accordion 
ch1Box = widgets.Text(description='Ch1 [dBm]:'); ch2Box = widgets.Text(description='Ch2 [dBm]:'); ch3Box = widgets.Text(description='Ch3 [dBm]:'); ch4Box = widgets.Text(description='Ch4 [dBm]:')
ch5Box = widgets.Text(description='Ch5 [dBm]:'); ch6Box = widgets.Text(description='Ch6 [dBm]:'); ch7Box = widgets.Text(description='Ch7 [dBm]:'); ch8Box = widgets.Text(description='Ch8 [dBm]:')
s0Box = widgets.Text(description='S0:'); s1Box = widgets.Text(description='S1:'); s2Box = widgets.Text(description='S2:'); s3Box = widgets.Text(description='S3:')

contMeasHTML = widgets.HTML(value='<b>Continuous Measurement of PM & PS: All Channel</b>')
contMeasButton = widgets.Button(description='Start', style={'description_width':'initial'}); contMeasStatusBox = widgets.Text()
contMeasTimeBox = widgets.FloatText(description='Measurement Time [hours]:', value=176, style={'description_width':'initial'})
chanBoxRawHandler = widgets.VBox([widgets.HBox([widgets.VBox([ch1Box, ch2Box, ch3Box, ch4Box]), 
                                                widgets.VBox([ch5Box, ch6Box, ch7Box, ch8Box]),
                                                widgets.VBox([s0Box, s1Box, s2Box, s3Box])]),
                                  widgets.VBox([contMeasHTML, contMeasTimeBox, 
                                                widgets.HBox([contMeasButton, contMeasStatusBox])])])

## third accordion
lavRefChanBox = widgets.Dropdown(description='Ref Chan.', options=['1', '2', '3', '4', '5', '6', '7', '8'], value='1', style={'description_width':'initial'}); lavPair1ChanBox = widgets.Dropdown(description='Pair1 Chan.', options=['1', '2', '3', '4', '5', '6', '7', '8'], value='2', style={'description_width':'initial'}); lavPair2ChanBox = widgets.Dropdown(description='Pair2 Chan.', options=['1', '2', '3', '4', '5', '6', '7', '8'], value='3', style={'description_width':'initial'})
lavRefBaseBox = widgets.Text(description='Ref Baseline [dBm]', value='0.000', style={'description_width':'initial'}); lavPair1BaseBox = widgets.Text(description='Pair1 Baseline [dBm]', value='0.000', style={'description_width':'initial'}); lavPair2BaseBox = widgets.Text(description='Pair2 Baseline [dBm]', value='0.000', style={'description_width':'initial'})
lavRefRawBox = widgets.Text(description='Ref (Curr.) [dBm]', value='0.000', style={'description_width':'initial'}); lavPair1RawBox = widgets.Text(description='Pair1 (Curr.) [dBm]', value='0.000', style={'description_width':'initial'}); lavPair2RawBox = widgets.Text(description='Pair2 (Curr.) [dBm]', value='0.000', style={'description_width':'initial'})
lavPair1LossFacetBox = widgets.Text(description='Pair1 Loss (Facet) [dB]', value='0.000', style={'description_width':'initial'}); lavPair2LossFacetBox = widgets.Text(description='Pair2 Loss (Facet) [dB]', value='0.000', style={'description_width':'initial'})
lavPairDiffBox = widgets.Text(description='Pair1-Pair2 [dB]', value='0.000', style={'description_width':'initial'})
lavSaveBaselineButton = widgets.Button(description='Save Baseline'); lavLoadBaselineButton = widgets.Button(description='Load Baseline'); lavFreezeButton = widgets.ToggleButton(description='Freeze', value=False)
lavECHandler = widgets.VBox([widgets.HBox([lavRefChanBox, lavPair1ChanBox, lavPair2ChanBox]),
                             widgets.HBox([lavRefBaseBox, lavPair1BaseBox, lavPair2BaseBox, lavSaveBaselineButton, lavLoadBaselineButton]), 
                             widgets.HBox([lavRefRawBox, lavPair1RawBox, lavPair2RawBox]), 
                             widgets.HBox([lavPair1LossFacetBox, lavPair2LossFacetBox, lavPairDiffBox, lavFreezeButton])])

## fourth accordion
barRefChanBox = widgets.Dropdown(description='Ref. Chan.', options=['1', '2', '3', '4', '5', '6', '7', '8'], value='1', style={'description_width':'initial'})
barMeasChanBox = widgets.Dropdown(description='Meas. Chan.', options=['1', '2', '3', '4', '5', '6', '7', '8'], value='2', style={'description_width':'initial'})
barPowerBaseBox = widgets.Text(description='Baseline [dBm]', value='0.000', style={'description_width':'initial'}); barPowerDynBaseBox = widgets.Text(description='dynBaseline [dBm]', value='0.000', style={'description_width':'initial'})
barPowerRawBox = widgets.Text(description='Power (Curr.) [dBm]', value='0.000', style={'description_width':'initial'}); barPowerDynRawBox = widgets.Text(description='dynBaseline (Curr.) [dBm]', value='0.000', style={'description_width':'initial'})
barLossBox = widgets.Text(description='Loss (Relative) [dB]', value='0.000', style={'description_width':'initial'}); barLossFacetBox = widgets.Text(description='Loss (Facet) [dB]', value='0.000', style={'description_width':'initial'})
barLossFacetDynBox = widgets.Text(description='Loss (Facet), dynCorrected [dB]', value='0.000', style={'description_width':'initial'})
barSaveBaselineButton = widgets.Button(description='Save Baseline'); barLoadBaselineButton = widgets.Button(description='Load Baseline'); barFreezeButton = widgets.ToggleButton(description='Freeze', value=False)
barECHandler = widgets.VBox([widgets.HBox([barRefChanBox, barMeasChanBox]), 
                             widgets.HBox([barPowerDynBaseBox, barPowerBaseBox, barSaveBaselineButton, barLoadBaselineButton]), 
                             widgets.HBox([barPowerDynRawBox, barPowerRawBox]),
                             widgets.HBox([barLossBox, barLossFacetBox, barLossFacetDynBox, barFreezeButton])])

## fifth accordion
barConnectLeftButton = widgets.Button(description='Connect Left Piezo(s)'); barConnectRightButton = widgets.Button(description='Connect Right Piezo(s)'); 
barOptiButton = widgets.Button(description='Optimize', disabled=True, style={'description_width':'initial'})
barOptiParamStep = widgets.BoundedFloatText(description='Step', value=0.1, min=0, step=0.05, style={'description_width':'initial'})
barOptiParamMaxSteps = widgets.BoundedIntText(description='Steps/Iter', value=30, min=1, step=1, style={'description_width':'initial'})
barOptiParamAlpha = widgets.BoundedIntText(description='α', value=50, min=0, step=1, style={'description_width':'initial'})
barOptiParamBeta = widgets.BoundedIntText(description='β', value=5, min=0, step=1, style={'description_width':'initial'})
barECOptLog = widgets.Textarea(layout=widgets.Layout(height='100px', width='100%'))
barECOptHandler = widgets.VBox([widgets.HBox([barConnectLeftButton, barConnectRightButton]), 
                                widgets.HBox([barOptiParamStep, barOptiParamMaxSteps, barOptiParamAlpha, barOptiParamBeta, barOptiButton]),
                                widgets.HBox([barECOptLog]),
                                widgets.HBox([barLossBox, barLossFacetBox, barLossFacetDynBox, barFreezeButton])])

## sixth accordion
sweepStartBox = widgets.BoundedFloatText(description='Start Wavelength [nm]', value=1490, min=1490, max=1640, step=0.0001, style={'description_width':'initial'})
sweepEndBox = widgets.BoundedFloatText(description='End Wavelength [nm]', value=1640, min=1490, max=1640, step=0.0001, style={'description_width':'initial'})
sweepSpeedBox = widgets.Dropdown(description='Sweep Speed [nm/s]', options=['0.5', '1.0', '2.0', '5.0', '10.0', '20.0', '40.0', '50.0'], value='10.0', style={'description_width':'initial'})
sweepChanBox = widgets.Dropdown(description='PM Chan.', options=['1', '2', '3', '4', '5', '6', '7', '8'], value='1', style={'description_width':'initial'})
sweepStepBox = widgets.BoundedFloatText(description='Sweep Step [nm]', value=0.01, min=0.0001, max=1000, step=0.0001, style={'description_width':'initial'})
runSweepButton = widgets.Button(description='Run Sweep', disabled=True); plotSweepButton = widgets.Button(description='Plot Sweep', disabled=True); saveSweepButton = widgets.Button(description='Save Sweep', disabled=True);
sweepStatusBox = widgets.Text(value='Idle...')
plotSweepBox = widgets.Output()
tlsSweepHandler = widgets.VBox([widgets.HBox([sweepStartBox, sweepEndBox, sweepStatusBox, runSweepButton, plotSweepButton]), 
                                   widgets.HBox([sweepSpeedBox, sweepStepBox, sweepChanBox, saveSweepButton]),
                                   widgets.HBox([plotSweepBox])])

## all accordion
AccordHandler = widgets.Accordion(children=[instConnHandler, chanBoxRawHandler, lavECHandler, barECHandler, barECOptHandler, tlsSweepHandler])
AccordHandler.set_title(0, 'Instruments Connection Check'); AccordHandler.set_title(1, 'PM & PS All Channel [Raw]')
AccordHandler.set_title(2, 'Lavender EC & GC: Facet Loss [Pair - Relative to Reference]'); AccordHandler.set_title(3, 'Barbados EC: Facet Loss')
AccordHandler.set_title(4, 'Barbados EC: Piezo Optimization'); AccordHandler.set_title(5, 'TLS & PM Wavelength Sweep')

## display widgets
def arthur_main():
    display(
        widgets.VBox([widgets.Label('AdvancedInterconnect: Optimized for Chariot [TLS N7778C] & Ultimae [TLS N7776C] with [PM N7745C] & [PS N7786C]'),
            widgets.HBox([instFindHandler, instStartHandler,
                        widgets.VBox([connectChariotButton, connectUltimaeButton]),
                        connectPSCheck]),
            AccordHandler]), 
        outputMain)
arthur_main()

# Clear cell output and redraw
def restart_cell():
    IPython.display.clear_output()
    arthur_main()

## handlers
def scanButtonHandler(val):
    with outputMain:
        dropdownBox.options = list(resMan.list_resources())
        print(str(datetime.now()) + '   Scanning for instruments...')
scanButton.on_click(scanButtonHandler)

def dropdownBoxHandler(val):
    global instAddress
    if val['type'] == 'change' and val['name'] == 'value':
        with outputMain:
            instAddress = val['new']
            print(str(datetime.now()) + '   Selected device address: ' + instAddress)
dropdownBox.observe(dropdownBoxHandler)

def checkConnectButtonHandler(val):
    global instHandler, instAddress
    with outputMain:
        connectInstMeasButton.disabled = True
        print(str(datetime.now()) + '   Connecting to: ' + instAddress)
        instHandler = resMan.open_resource(instAddress)
        instIDBox.value = instHandler.query('*IDN?').strip()
        print(str(datetime.now()) + '   Connected to: ' + instIDBox.value)
        connectInstMeasButton.disabled = False
checkConnectButton.on_click(checkConnectButtonHandler)

def connectInstButtonHandler(val):
    global instHandler, pmHandler, tlsHandler, psHandler, instAddress, pmPortCount
    with outputMain:
        if instIDBox.value != '':
            if instAddress == pmAddrChariot or pmAddrUltimae:
                pmHandler = resMan.open_resource(instAddress)
                print(str(datetime.now()) + '   Connection established to PD: ' + instIDBox.value); instIDBox.value = ''
                pmPortCount = pmHandler.query('FETC:POW:ALL:CSV?').count(',') + 1
                for i in range(pmPortCount): 
                    pmHandler.write(':SENS' + str(i+1) + ':POW:UNIT 0') ## dBm default
                    pmHandler.write(':SENS' + str(i+1) + ':POW:WAV 1550NM') ## 1550nm default
                    pmHandler.write(':INIT' + str(i+1) + ':CONT 1') ## turn on continuous measurement
                    pmHandler.write(':SENS' + str(i+1) + ':FUNC:STAT LOGG,STOP') ## stop logging for all channel

            elif instAddress == psAddr:
                psHandler = resMan.open_resource(instAddress)
                print(str(datetime.now()) + '   Connection established to PS: ' + instIDBox.value); instIDBox.value = ''
                psHandler.write(':POL:WAV 1.550E-6') ## 1550nm default
                psHandler.write(':POL:POW:UNIT 0') ## dBm default

            elif instAddress == tlsAddrChariot or tlsAddrUltimae:
                tlsHandler = resMan.open_resource(instAddress)
                tlsHandler.write(':SOUR0:WAV 1550NM') ## 1550nm default
                print(str(datetime.now()) + '   Connection established to TLS: ' + instIDBox.value); instIDBox.value = ''

            else:
                instHandler = resMan.open_resource(instAddress)
                print(str(datetime.now()) + '   Connection established to: ' + instIDBox.value); instIDBox.value = ''
        else:
            instIDBox.value = 'Check connection first!'
connectInstMeasButton.on_click(connectInstButtonHandler)

def stopMeasButtonHandler(val):
    global instHandler, pmHandler, psHandler, tlsHandler, instAddress, sweepStatus
    with outputMain:
        print(str(datetime.now()) + '   Ending measurements...')
        instIDBox.value = ''; runSweepButton.disabled = True
        # tlsHandler.close(); pmHandler.close()
        instHandler = 'None'; tlsHandler = 'None'; pmHandler = 'None'; psHandler = 'None'; sweepStatus = 'Idle'

        tlsConnBox.value = 'Not Connected'; pmConnBox.value = 'Not Connected'; psConnBox.value = 'Not Connected'; sweepStatusBox.value = 'Idle...'; contMeasStatusBox.value = 'Idle...'
        print(str(datetime.now()) + '   Device disconnected.')
stopMeasButton.on_click(stopMeasButtonHandler)

def connectUltimaeButtonHandler(val):
    global connStatus, instHandler, pmHandler, tlsHandler, psHandler, instAddress, pmPortCount, tlsAddrUltimae, pmAddrUltimae, psAddr
    with outputMain:
        connStatus = 'Connecting'

        while instIDBox.value == '':
            tlsHandler = resMan.open_resource(tlsAddrUltimae); instIDBox.value = tlsHandler.query('*IDN?').strip()
        print(str(datetime.now()) + '   Connection established to TLS: ' + instIDBox.value); instIDBox.value = ''
        tlsHandler.write(':SOUR0:WAV 1550NM') ## 1550nm default
        tlsHandler.timeout = 10000
        temp = int(tlsHandler.query('*OPC?'))
        while temp == 0: time.sleep(0.01); temp = int(tlsHandler.query('*OPC?'))

        if connectPSCheck.value:
            psHandler = resMan.open_resource(psAddr); instIDBox.value = psHandler.query('*IDN?').strip()
            print(str(datetime.now()) + '   Connection established to PS: ' + instIDBox.value); instIDBox.value = ''
            psHandler.write(':POL:WAV 1.550E-6') ## 1550nm default
            psHandler.write(':POL:POW:UNIT 0') ## dBm default
            psHandler.timeout = 10000
            time.sleep(1)
            temp = int(psHandler.query('*OPC?'))
            while temp == 0: time.sleep(0.01); temp = int(psHandler.query('*OPC?'))

        pmHandler = resMan.open_resource(pmAddrUltimae); instIDBox.value = pmHandler.query('*IDN?').strip()
        print(str(datetime.now()) + '   Connection established to PD: ' + instIDBox.value); instIDBox.value = ''
        pmPortCount = pmHandler.query('FETC:POW:ALL:CSV?').count(',') + 1
        for i in range(pmPortCount): 
            pmHandler.write(':SENS' + str(i+1) + ':POW:UNIT 0') ## dBm default
            pmHandler.write(':SENS' + str(i+1) + ':POW:WAV 1550NM') ## 1550nm default
            pmHandler.write(':INIT' + str(i+1) + ':CONT 1') ## turn on continuous measurement
            pmHandler.write(':SENS' + str(i+1) + ':FUNC:STAT LOGG,STOP') ## stop logging for all channel
        pmHandler.timeout = 10000
        time.sleep(1)
        temp = int(pmHandler.query('*OPC?'))
        while temp == 0: time.sleep(0.01); temp = int(pmHandler.query('*OPC?'))

        if switchAddr:
            switchHandler = resMan.open_resource(switchAddr); instIDBox.value = switchAddr.query('*IDN?').strip()
            print(str(datetime.now()) + '   Connection established to PD: ' + instIDBox.value); instIDBox.value = ''
            switchHandler.timeout = 10000
            time.sleep(1)
            temp = int(switchHandler.query('*OPC?'))
            while temp == 0: time.sleep(0.01); temp = int(switchHandler.query('*OPC?'))

        barLoadBaselineButtonHandler(1)
        instIDBox.value = 'Ultimae setup connected!'
        print(str(datetime.now()) + '   Ultimae setup connected!')
        connStatus = 'Idle'
connectUltimaeButton.on_click(connectUltimaeButtonHandler)

def connectChariotButtonHandler(val):
    global connStatus, instHandler, pmHandler, tlsHandler, psHandler, instAddress, pmPortCount, tlsAddrChariot, pmAddrChariot, psAddr
    with outputMain:
        connStatus = 'Connecting'

        while instIDBox.value == '':
            tlsHandler = resMan.open_resource(tlsAddrChariot); instIDBox.value = tlsHandler.query('*IDN?').strip()
        print(str(datetime.now()) + '   Connection established to TLS: ' + instIDBox.value); instIDBox.value = ''
        tlsHandler.write(':SOUR0:WAV 1550NM') ## 1550nm default
        tlsHandler.timeout = 10000
        temp = int(tlsHandler.query('*OPC?'))
        while temp == 0: time.sleep(0.01); temp = int(tlsHandler.query('*OPC?'))

        if connectPSCheck.value:
            psHandler = resMan.open_resource(psAddr); instIDBox.value = psHandler.query('*IDN?').strip()
            print(str(datetime.now()) + '   Connection established to PS: ' + instIDBox.value); instIDBox.value = ''
            psHandler.write(':POL:WAV 1.550E-6') ## 1550nm default
            psHandler.write(':POL:POW:UNIT 0') ## dBm default
            psHandler.timeout = 10000
            time.sleep(1)
            temp = int(psHandler.query('*OPC?'))
            while temp == 0: time.sleep(0.01); temp = int(psHandler.query('*OPC?'))

        pmHandler = resMan.open_resource(pmAddrChariot); instIDBox.value = pmHandler.query('*IDN?').strip()
        print(str(datetime.now()) + '   Connection established to PD: ' + instIDBox.value); instIDBox.value = ''
        pmPortCount = pmHandler.query('FETC:POW:ALL:CSV?').count(',') + 1
        for i in range(pmPortCount): 
            pmHandler.write(':SENS' + str(i+1) + ':POW:UNIT 0') ## dBm default
            pmHandler.write(':SENS' + str(i+1) + ':POW:WAV 1550NM') ## 1550nm default
            pmHandler.write(':INIT' + str(i+1) + ':CONT 1') ## turn on continuous measurement
            pmHandler.write(':SENS' + str(i+1) + ':FUNC:STAT LOGG,STOP') ## stop logging for all channel
        pmHandler.timeout = 10000
        time.sleep(1)
        temp = int(pmHandler.query('*OPC?'))
        while temp == 0: time.sleep(0.01); temp = int(pmHandler.query('*OPC?'))

        instIDBox.value = 'Chariot setup connected!'
        print(str(datetime.now()) + '   Chariot setup connected!')
        connStatus = 'Idle'
connectChariotButton.on_click(connectChariotButtonHandler)

def contMeasButtonHandler(val):
    global contMeasStatus, dataTransReal, dataPolReal
    with outputMain:
        if contMeasStatus != 0: 
            contMeasStatus = 0
        else: 
            print(str(datetime.now()) + '   Continuous measurement started...')
            contMeasStatus = 1 
            fileName = time.strftime("%y%m%d-%H%M%S") + '.csv'; timeInit = time.time_ns()

        while True:
            contMeasButton.disabled = True
            timeDeltaCurr = round(1e-9*(time.time_ns()-timeInit),9)
            if timeDeltaCurr/3600 > contMeasTimeBox.value: break
            
            if contMeasStatus == 1:
                contMeasStatusBox.value = 'Running...'

                temp = [timeDeltaCurr]
                if pmHandler != 'None': temp = temp + dataTransReal[:] ## power meter
                if psHandler != 'None': temp = temp + dataPolReal[:] ## polarization synthesizer

                if switchHandler != 'None':
                    if round(timeDeltaCurr/60) % 15:

                        currSwitchConfig = switchHandler.query(':ROUT1:CONF:ROUT')
                        ##change switch
                        switchHandler.write(':ROUT1: A,2')
                        time.sleep(1)

                if os.path.isfile(fileName): writeMode = 'a'
                else: writeMode = 'w'

                with open(fileName, writeMode, newline='') as file:
                    dataWriter = csv.writer(file, delimiter=',')
                    dataWriter.writerow(temp)
                    time.sleep(0.5)
            else: contMeasStatusBox.value = 'Idle...'; contMeasButton.disabled = False; break
contMeasButton.on_click(contMeasButtonHandler)

def lavSaveBaselineButtonHandler(val):
    with outputMain:
        lavRefBaseBox.value = lavRefRawBox.value
        lavPair1BaseBox.value = str(round(float(lavPair1RawBox.value),3))
        lavPair2BaseBox.value = str(round(float(lavPair2RawBox.value),3))
        with open('ChariotBaseline.txt', 'a') as file:
            file.write(f'{time.ctime()},\t{lavRefBaseBox.value},\t{lavPair1BaseBox.value},\t{lavPair2BaseBox.value}\n')
lavSaveBaselineButton.on_click(lavSaveBaselineButtonHandler)

def lavLoadBaselineButtonHandler(val):
    with outputMain:
        with open('ChariotBaseline.txt', 'r') as file:
            lavRefBaseBox.value = file.readlines()[-1].split(',')[1].strip()
        with open('ChariotBaseline.txt', 'r') as file:
            lavPair1BaseBox.value = file.readlines()[-1].split(',')[2].strip()
        with open('ChariotBaseline.txt', 'r') as file:
            lavPair2BaseBox.value = file.readlines()[-1].split(',')[3].strip()
lavLoadBaselineButton.on_click(lavLoadBaselineButtonHandler)

def barSaveBaselineButtonHandler(val):
    with outputMain:
        barPowerBaseBox.value = barPowerRawBox.value
        barPowerDynBaseBox.value = str(round(dataTransReal[int(barRefChanBox.value)-1],3))
        with open('UltimaeBaseline.txt', 'a') as file:
            file.write(f'{time.ctime()},\t{barPowerRawBox.value},\t{str(round(dataTransReal[int(barRefChanBox.value)-1],3))}\n')
barSaveBaselineButton.on_click(barSaveBaselineButtonHandler)

def barLoadBaselineButtonHandler(val):
    with outputMain:
        with open('UltimaeBaseline.txt', 'r') as file:
            barPowerDynBaseBox.value = file.readlines()[-1].split(',')[2].strip()
        with open('UltimaeBaseline.txt', 'r') as file:
            barPowerBaseBox.value = file.readlines()[-1].split(',')[1].strip()
barLoadBaselineButton.on_click(barLoadBaselineButtonHandler)

# Miscellaneous
def log(message):
    #print(str(datetime.now()) + '   ' + message)
    barECOptLog.value = str(message) + '\n' + barECOptLog.value

nanoTrak = None
piezo = None
strainGauge = None
pp = None

def barConnectButtonHandler(val, nt=leftNanoTrakAddr, pz=leftPiezoAddr, sg=leftStrainGaugeAddr):
    global nanoTrak, strainGauge, piezo, pp
    with outputMain:
        barOptiButton.disabled = True
        #pp = None # Delete any ProgressPlot
        deviceListSize = KCubeDeviceListSize()

        if deviceListSize >= 3:
            log('Trying to connect to piezos and sensor...')    
            nanoTrak = NanoTrakController(nt)
            piezo = PiezoController(pz)
            strainGauge = StrainGaugeController(sg)
            if nanoTrak.status + piezo.status + strainGauge.status == 0:
                ## poll all
                nanoTrak.startPolling()
                strainGauge.startPolling()
                piezo.startPolling()
                ## load default settings
                strainGauge.loadSettings()
                log('Strain gauge default setting loaded...')

                ## nanotrak: latch and set home pos
                pos_home = NT_HVComponent(NTUToBit(5), NTUToBit(5))
                nanoTrak.setMode('latch')
                nanoTrak.setCircleHomePosition(pos_home)
                log('NanoTrak latched and home position set...')

                ## strain gauge: enable and zero based on piezo zero position
                strainGauge.enable()
                #piezo.setControlMode('open')
                #piezo.enable()
                #piezo.setZeroVoltage()
                #strainGauge.setZero()
                strainGauge.waitForReady()
                log('Strain gauge is enabled and zeroed...')

                ## check set piezo to close loop
                piezo.setControlMode('closed')
                log('Piezo set to closed loop...')

                ## test piezo travel (ONLY UNCOMMENT IF FIBER IS FAR FROM CHIP EDGE)
                #log('Testing x positioning')
                #for _ in range(3):
                #    piezo.setPos(0); piezo.setPos(100)
                #piezo.setPos(0)

                ## enable piezo, then check set piezo to 20%
                #PiezoLeft.PCC_Enable(leftPiezoAddr); time.sleep(3)
                # temp = piezo.getPos(); log('first get post: ' + str(temp))
                #if piezo.getPos() < 10:
                #    piezo.setPos(10)
                #    time.sleep(1)
                log('Piezo enabled and current piezo X position is [%]: ' + str(piezo.getPos()))

                ## nanotrak: init
                # currPos = NT_HVComponent(0, 0); NanoTrakLeft.NT_GetCirclePosition(leftNanoTrakAddr, byref(currPos)) ## get curr pos
                # currCirPar = NT_CircleParameters(0,0,0,0,0,0); NanoTrakLeft.NT_GetCircleParams(leftNanoTrakAddr, byref(currCirPar)) ## get curr circ params
                nanoTrak.setCircleDiameter(0.75) ## set circ dia to 0.75 NTU
                log('NanoTrak is set and current circle diameter is [NTU]: ' + str(nanoTrak.getCircleDiameter()))

                nanoTrak.setMode('track', 2); nanoTrak.setMode('latch')
                log(f'Piezos connected, configured, track and latched!')
                barOptiButton.disabled = False
            else: log(f'Cannot connect to piezos! -- {nanoTrak.status}, {piezo.status}, {strainGauge.status}')
        else: log('Cannot find piezos! (incorrect number of devices)')
barConnectLeftButton.on_click(functools.partial(barConnectButtonHandler, nt=leftNanoTrakAddr,pz=leftPiezoAddr,sg=leftStrainGaugeAddr))
barConnectRightButton.on_click(functools.partial(barConnectButtonHandler, nt=rightNanoTrakAddr,pz=rightPiezoAddr,sg=rightStrainGaugeAddr))

def barOptiButtonHandler(val):
    global nanoTrak, strainGauge, piezo

    def currentPower():
        return round(float(barLossFacetDynBox.value)*1000,3)

    restart_cell()
    time.sleep(0.5)
    pp = ProgressPlot(plot_names=['Loss/EC'], x_label='Step', y_lim=[0,-currentPower()/1000])
    with outputMain:
        # Step size by which to increment the fiber x-position
        step = barOptiParamStep.value # 0.1
        # Maximum number of steps before tracking
        maxSteps = barOptiParamMaxSteps.value # 30
        # Paramater that defines tolerance for minor power increases
        alpha = barOptiParamAlpha.value # 50
        # Minimum power change after several steps to determine if contact is made
        beta = barOptiParamBeta.value # 5

        nanoTrak.setMode('track', 2)
        nanoTrak.setMode('latch')
        time.sleep(2)

        # Sentinel to stop fiber movement when physical contact is made with chip
        makingContact = False
        # Move fiber inwards (x), tracking alignment plane (y-z) occasionally
        while not makingContact:
            log('Entering loop')
            # Initialize variables used to track changes while fiber moves
            totalMovement = 0
            prevPower = currentPower()
            # Continue moving fiber inwards by step size
            while not makingContact and totalMovement < maxSteps:
                # Get starting conditions
                currPos = piezo.getPos()
                initPow = currentPower()

                # Move to current position + step size
                piezo.setPos(currPos+step)
                totalMovement += 1

                log(f'Current position before moving: {currPos:0.3f}%, {initPow/1000:0.3f} dB, Δ={initPow-currentPower()}')
                pp.update(-initPow/1000)
                ## If loss is getting higher, stop
                if initPow-currentPower() >= alpha:
                    makingContact = True
                    log(f'Loss getting higher {initPow} -> {currentPower()}. STOP!')

                # If power is remaining stable after several steps, stop
                if totalMovement % (maxSteps/4) == 0:
                    if currentPower()-prevPower <= beta:
                        nanoTrak.setMode('track', 2); nanoTrak.setMode('latch'); time.sleep(3)
                        if currentPower()-prevPower <= beta:
                            makingContact = True
                            log('Power remaining stable after several steps. STOP!')
                        else:
                            log('Stopped to check if power is leveling off, but it seems fine after tracking.')
                    else:
                        prevPower = currentPower()

            # Track and latch NanoTrak to re-optimize Y-Z position
            #  checking that circle position doesn't drift rapidly indicating contact
            log('Tracking now')
            nanoTrak.setMode('track', 2)
            #if checkCircleRunaway(): # <----------- IMPLEMENT THIS
            #    makingContact = True
            nanoTrak.setMode('latch')
            time.sleep(3)

        # Done optimizing
        optimizedPower = currentPower()

        #piezo.setPos(20) ## but set back to 20%
        strainGauge.disable(); strainGauge.close(); ## disable and close 
        piezo.close() ## disable and close
        nanoTrak.setMode('latch'); nanoTrak.close() ## latch and close
        log('Everything is closed! for now...')
barOptiButton.on_click(barOptiButtonHandler)
# def barOptiLeftButtonHandler(val):
#     global nanoTrak, strainGauge, piezo
#     with outputMain:
#         temp = 0.01
#         currPos = piezo.getPos()
#         print(f'curr pos: {piezo.getPos()}')
#         time.sleep(1)

#         initPow = round(float(barLossFacetDynBox.value)*1000,3); currPow = initPow
#         print(f'curr power: {initPow} mdB')

#         while currPow < -600:
#             print(f'moving to: {currPos+temp}')
#             piezo.setPos(currPos+temp) ## move
#             time.sleep(0.5)

#             currPow = round(float(barLossFacetDynBox.value)*1000,3)
#             print(f'curr power: {currPow} mdB')
#             print(f'curr pos: {piezo.getPos()}')
#             ## If loss is getting higher, break
#             if initPow-currPow >= 0: break 
#             temp += temp

#         print('should be less than 600mdB! if not, do something')
#         nanoTrak.setMode('track', 2); nanoTrak.setMode('latch')
#         print('tracked and latched!')

#         print('start next steps!')
#         ## sam will continue this part

#         print('resetting...')
#         print(round(float(barLossFacetDynBox.value),3))
#         # while round(float(barLossFacetDynBox.value)*1000,3) > -1000:
#         #     PiezoLeft.PCC_SetPosition(leftPiezoAddr, perToBit(currPos-temp))
#         #     temp -= temp
#         print('should be higher than 1000mdB! safe to move the chip now!')
#         print(f'curr pos: {piezo.getPos()}')

#         piezo.setControlMode('closed') ## stay in closed loop
#         piezo.setPos(20) ## but set back to 20%
#         strainGauge.disable(); strainGauge.close(); ## disable and close 
#         piezo.disable(); piezo.close() ## disable and close
#         nanoTrak.setMode('latch'); nanoTrak.close() ## latch and close
#         log('Everything is closed! for now...')
# barOptiLeftButton.on_click(barOptiLeftButtonHandler)

def runSweepButtonHandler(val):
    global sweepStatus, dataLambda, dataTrans, pmPortCount, dataSweep
    with outputMain:
        ## init params
        chanPick = [int(sweepChanBox.value)]
        sweepStatusBox.value = 'Sweeping...'; sweepStatus = 'Sweeping'
        sweepLambdaLength = sweepEndBox.value-sweepStartBox.value
        expectedTrigger = round(sweepLambdaLength/sweepStepBox.value+1); # print('Calculated Expected Trigger: ' + str(expectedTrigger))
        runSweepButton.disabled = True

        ## check and set data points
        if len(chanPick) == 1:
            expectedTrigger = 10001
            expectedSweepStep = sweepLambdaLength/(expectedTrigger-1)
        elif len(chanPick) == 2:
            expectedTrigger = 8001
            expectedSweepStep = sweepLambdaLength/(expectedTrigger-1)
        elif len(chanPick) >=3: 
            print(str(datetime.now()) + '   Maximum number of supported channel: 2. Exiting sweep....')
            return
        
        if sweepStepBox.max >= expectedSweepStep >= sweepStepBox.min:
            sweepStepBox.value = expectedSweepStep
        elif expectedSweepStep < sweepStepBox.min: sweepStepBox.value = sweepStepBox.min
        else: sweepStepBox.value = sweepStepBox.max

        ## set TLS parameter
        print(str(datetime.now()) + '   Setting TLS parameters...')
        tlsHandler.write(':TRIG0:CONF DEF') ## trigger default
        tlsHandler.write(':SOUR0:POW:STAT 1') ## set laser on
        tlsHandler.write(':TRIG0:INP SWS') ## trigger input as wavelength start
        tlsHandler.write(':TRIG0:OUTP STF') ## trigger outputMain as step finished
        tlsHandler.write(':SOUR0:WAV ' + str(round(sweepStartBox.value, 4)) + 'NM') ## set wavelength to start of the sweep
        time.sleep(2) ## wait for laser to settle

        tlsHandler.write(':SOUR0:AM:STAT OFF') ## turn off amplitude modulation
        tlsHandler.write(':SOUR0:WAV:SWE:MODE CONT') ## continuous sweep
        tlsHandler.write(':SOUR0:WAV:SWE:REP ONEW') ## oneway sweep
        tlsHandler.write(':SOUR0:WAV:SWE:CYCL 1') ## one cycle
        tlsHandler.write(':SOUR0:WAV:SWE:LLOG 1') ## lambda logging
        tlsHandler.write(':SOUR0:WAV:SWE:STAR ' + str(sweepStartBox.value) + 'NM') ## start wavelength
        tlsHandler.write(':SOUR0:WAV:SWE:STOP ' + str(sweepEndBox.value) + 'NM') ## stop wavelength
        tlsHandler.write(':SOUR0:WAV:SWE:SPE ' + str(sweepSpeedBox.value) + 'NM/S') ## sweep speed
        tlsHandler.write(':SOUR0:WAV:SWE:STEP ' + str(sweepStepBox.value) + 'NM') ## sweep step
        # expectedTrigger = round(float(tlsHandler.query(':SOUR0:WAV:SWE:EXP?').strip())) ## get expected trigger number
        # print('Instrument Expected Trigger: ' + str(int(tlsHandler.query(':SOUR0:WAV:SWE:EXP?').strip()))) ## print expected trigger from instrument
        
        ## set PM parameter
        print(str(datetime.now()) + '   TLS parameters set. Setting PD parameters...')
        # for i in range(pmPortCount): pmHandler.write(':SENS' + str(i+1) + ':FUNC:STAT LOGG,STOP') ## stop logging for all channel
        for i in range(len(chanPick)): pmHandler.write(':SENS' + str(chanPick[i]) + ':FUNC:STAT LOGG,STOP') ## stop logging for picked channel

        for i in range(len(chanPick)): ## for chosen channel(s), do
            pmHandler.write(':TRIG' + str(chanPick[i]) + ':CONF DEF') ## trigger default
            pmHandler.write(':TRIG' + str(chanPick[i]) + ':INP SME') ## trigger input as single measurement every trigger
            pmHandler.write(':SENS' + str(chanPick[i]) + ':POW:GAIN:AUTO 0') ## auto gain off
            pmHandler.write(':SENS' + str(chanPick[i]) + ':POW:RANG:AUTO 0') ## auto range off
            pmHandler.write(':SENS' + str(chanPick[i]) + ':POW:RANG 10DBM') ## lowest power range that will not clip the strongest signal
            pmHandler.write(':SENS' + str(chanPick[i]) + ':POW:UNIT 0') ## dBm unit
            pmHandler.write(':SENS' + str(chanPick[i]) + ':POW:WAV 1550NM') ## wavelength to 1550
            pmHandler.write(':SENS' + str(chanPick[i]) + ':FUNC:PAR:LOGG ' + str(expectedTrigger) + ',100US') ## parameter for logging function
        temp = int(pmHandler.query('*OPC?'))
        while temp == 0: time.sleep(0.01); temp = int(pmHandler.query('*OPC?'))

        ## parameter all set, run the sweep
        print(str(datetime.now()) + '   PD parameters set. Running wavelength sweep...')
        tlsHandler.write(':SOUR0:WAV:SWE 1') ## activate the sweep function
        time.sleep(1)
        
        tlsHandler.write(':SOUR0:WAV:SWE:SOFT') ## soft trigger
        tlsStatus = tlsHandler.query(':SOUR0:WAV:SWE:FLAG?').strip(); tlsOngoingStatus = '+1' ## TLS status
        while True:
            if tlsStatus != tlsOngoingStatus:
                tlsStatus = tlsHandler.query(':SOUR0:WAV:SWE:FLAG?').strip()
            else: break

        for i in range(len(chanPick)): pmHandler.write(':SENS' + str(chanPick[i]) + ':FUNC:STAT LOGG,STAR') ## set input trigger mode to single meas, one sample per trigger
        timeInit = time.time_ns()

        pmStatus = pmHandler.query(':SENS' + str(chanPick[0]) + ':FUNC:STAT?').strip(); pdOngoingStatus = pmStatus ## PM status
        while True:
            # print(tlsStatus + ' ' + tlsOngoingStatus); 
            if tlsStatus == tlsOngoingStatus:
                # pmStatus = pmHandler.query(':SENS' + str(chanPick[0]) + ':FUNC:STAT?').strip(); 
                tlsStatus = tlsHandler.query(':SOUR0:WAV:SWE:FLAG?').strip()
                # print(pmStatus + ' ' + tlsStatus); time.sleep(2)
            else:
                timeElapsed = 1e-9*(time.time_ns()-timeInit)

                ## get ALL the DATA
                dataLambda = tlsHandler.query_binary_values('SOUR0:READ:DATA? LLOG', datatype=u'd', container=np.array)
                for i in range(len(chanPick)): 
                    dataTrans = pmHandler.query_binary_values(':SENS' + str(chanPick[i]) + ':FUNC:RES?', container=np.array); time.sleep(0.5)
                # print(str(len(dataLambda)) + ' ' + str(len(dataTrans)))

                for i in range(len(chanPick)):
                    pmHandler.write(':SENS' + str(chanPick[i]) + ':POW:UNIT 0') ## dBm unit
                    pmHandler.write(':SENS' + str(chanPick[i]) + ':POW:WAV 1550NM') ## wavelength to 1550
                    pmHandler.write(':INIT' + str(chanPick[i]) + ':CONT 1') ## turn on continuous measurement
                    pmHandler.write(':SENS' + str(chanPick[i]) + ':FUNC:STAT LOGG,STOP') ## stop logging for all channel

                tlsHandler.write(':SOUR0:WAV 1550NM') ## set wavelength back to 1550nm
                tlsHandler.write(':SOUR0:POW 0DBM') ## set power to 0dBm
                tlsHandler.write(':TRIG0:CONF PASS') ## trigger default back to PASS

                dataLambdaExport = []
                for i in range(len(dataLambda)): 
                    dataLambdaExport.append(float(dataLambda[i])*1e9)
                dataSweep = pd.DataFrame(zip(dataLambdaExport, dataTrans))
                dataSweep.columns = ['Wavelength [nm]', 'Transmission [dBm]']

                time.sleep(2)
                tlsStatus = ''; pmStatus = ''; tlsOngoingStatus = ''; pdOngoingStatus = ''
                print(str(datetime.now()) + '   Wavelength sweep is done! It took: ' + str(round(timeElapsed, 3)) + 's')
                runSweepButton.disabled = False; sweepStatusBox.value = 'Sweep done! Idle...'; sweepStatus = 'Idle'

                plotSweepButtonHandler(1) ## plot
                break
runSweepButton.on_click(runSweepButtonHandler)

def plotSweepButtonHandler(val):
    global dataLambda, dataTrans, dataSweep

    dataLambdaExport = []
    for i in range(len(dataLambda)): 
        dataLambdaExport.append(float(dataLambda[i])*1e9)
    dataSweep = pd.DataFrame(zip(dataLambdaExport, dataTrans))
    dataSweep.columns = ['Wavelength [nm]', 'Transmission [dBm]']

    with plotSweepBox:
        plt.close('all')
        plotSweepBox.clear_output()
        figHandler, axHandler = plt.subplots(1, 1, layout='constrained')
        plt.plot(dataSweep['Wavelength [nm]'], dataSweep['Transmission [dBm]'], '-.k')
        plt.grid(True); plt.xlabel('Wavelength [nm]'); plt.ylabel('Transmission [dBm]')
        plt.xlim([sweepStartBox.value, sweepEndBox.value]); 
        plt.show()
plotSweepButton.on_click(plotSweepButtonHandler)

def saveSweepButtonHandler(val):
    global dataLambda, dataTrans, dataSweep

    timeString = time.strftime("%y%m%d-%H%M%S")
    dataSweep.to_csv(timeString + '.csv', index=False, header=True)
saveSweepButton.on_click(saveSweepButtonHandler)

## readout functions
def updateReadout():
    global cpuVal, dataTransReal, dataPolReal, sweepStatus, contMeasStatus, connStatus
    while True:
        lavFreezeStatus = lavFreezeButton.value; barFreezeStatus = barFreezeButton.value
        if tlsHandler != 'None': tlsConnBox.value = 'Connected'
        if pmHandler != 'None': pmConnBox.value = 'Connected'
        if psHandler != 'None': psConnBox.value = 'Connected'

        if connStatus == 'Idle' and sweepStatus == 'Idle':
            if psConnBox.value == 'Connected':
                if contMeasStatus != 1: contMeasStatusBox.value = 'Idle...'
                dataPolReal = psHandler.query(':POL:SOP?').strip().split(',')
                if dataPolReal[0] != '':
                    for i in range(len(dataPolReal)): dataPolReal[i] = float(dataPolReal[i])
                    s0Box.value = str(dataPolReal[0]); s1Box.value = str(dataPolReal[1]); s2Box.value = str(dataPolReal[2]); s3Box.value = str(dataPolReal[3])

            if pmConnBox.value == 'Connected':
                dataTransReal = pmHandler.query(':READ:CHAN:POW:ALL:CSV?').strip().split(',')
                if dataTransReal[0] != '':
                    while len(dataTransReal)<8:
                        dataTransReal.append(-200.000)

                    for i in range(len(dataTransReal)): dataTransReal[i] = float(dataTransReal[i]) ## add if the PM doesnt have 8 channels, set the rest of the values to -200
                    ch1Box.value = str(round(dataTransReal[0],3)); ch2Box.value = str(round(dataTransReal[1],3)); ch3Box.value = str(round(dataTransReal[2],3)); ch4Box.value = str(round(dataTransReal[3],3))
                    ch5Box.value = str(round(dataTransReal[4],3)); ch6Box.value = str(round(dataTransReal[5],3)); ch7Box.value = str(round(dataTransReal[6],3)); ch8Box.value = str(round(dataTransReal[7],3))
                    
                    if lavFreezeStatus == False:
                        lavRefRawBox.value = str(round(dataTransReal[int(lavRefChanBox.value)-1],3))
                        lavPair1RawBox.value = str(round(dataTransReal[int(lavPair1ChanBox.value)-1],3))
                        lavPair2RawBox.value = str(round(dataTransReal[int(lavPair2ChanBox.value)-1],3))
                        lavPair1LossFacetBox.value = str(round((float(lavPair1RawBox.value)-float(lavPair1BaseBox.value)-float(lavRefRawBox.value)+float(lavRefBaseBox.value))/2,3))
                        lavPair2LossFacetBox.value = str(round((float(lavPair2RawBox.value)-float(lavPair2BaseBox.value)-float(lavRefRawBox.value)+float(lavRefBaseBox.value))/2,3))
                        lavPairDiffBox.value = str(round(float(lavPair1LossFacetBox.value)-float(lavPair2LossFacetBox.value),3))

                    if barFreezeStatus == False:
                        barPowerRawBox.value = str(round(dataTransReal[int(barMeasChanBox.value)-1],3))
                        barPowerDynRawBox.value = str(round(dataTransReal[int(barRefChanBox.value)-1],3))
                        barLossBox.value = str(round(float(barPowerRawBox.value)-float(barPowerBaseBox.value),3))
                        barLossFacetBox.value = str(round((float(barPowerRawBox.value)-float(barPowerBaseBox.value))/2,3))
                        barLossFacetDynBox.value = str(round((float(barPowerRawBox.value)-float(barPowerBaseBox.value)-float(barPowerDynRawBox.value)+float(barPowerDynBaseBox.value))/2,3))

            if tlsConnBox.value == 'Connected' and pmConnBox.value == 'Connected':
                cpuVal = psutil.cpu_percent(); cpuUsageBox.value = f'{float(cpuVal):0.3f}'
                if sweepStatus == 'Idle': runSweepButton.disabled = False; plotSweepButton.disabled = False; saveSweepButton.disabled = False
            else: cpuUsageBox.value = ''

        time.sleep(0.1)

## threadings
threadsHandler = [] 
threadData = threading.Thread(target=updateReadout); threadsHandler.append(threadData)

for i in threadsHandler:
    i.start()

Output()